## Z Calibration Curves for 3D-DAOSTORM (and sCMOS).

In this example we are trying to determine the coefficients $w_o,c,d,A,B,C,D$ in this equation:

\begin{equation*}
W_{x,y} = w_o \sqrt{1 + \left(\frac{z-c}{d}\right)^2 + A\left(\frac{z-c}{d}\right)^3 + B\left(\frac{z-c}{d}\right)^4 + C\left(\frac{z-c}{d}\right)^5 + D\left(\frac{z-c}{d}\right)^6}
\end{equation*}

This is a modified form of a typical microscope defocusing curve. $W_x$, $W_y$ are the widths of the localization as measured by 3D-DAOSTORM and $z$ is the localization $z$ offset in $um$.

See also [Huang et al, Science, 2008](http://dx.doi.org/10.1126/science.1153529).


### Configuration

To perform z-calibration you need a movie of (small) fluorescent beads or single blinking dye molecules on a flat surface such as a coverslip. You then scan the coverslip through the focus of the microscope while recording a movie.

In this example we'll simulate blinking dyes on a coverslip. The PSF is created using the pupil function approach and is purely astigmatic.

Create an empty directory and change to that directory.

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate sample data for this example.

In [ ]:
import storm_analysis.jupyter_examples.dao3d_zcal as dao3d_zcal
dao3d_zcal.configure()

### 3D-DAOSTORM analysis of the calibration movie

Set parameters for 3D-DAOSTORM analysis. Note the analysis is done using the `3d` PSF model, a Gaussian with independent widths in X/Y.

In [ ]:
import storm_analysis.sa_library.parameters as params

# Load the parameters.
daop = params.ParametersDAO().initFromFile("example.xml")

# Set for a single iteration, we don't want multiple iterations of peak finding
# as this could cause stretched peaks to get split in half.
daop.changeAttr("iterations", 1)

# Use a large find max radius. This also reduces peak splitting.
daop.changeAttr("find_max_radius", 10)

# Use a higher threshold so that we don't get the dimmer localizations.
daop.changeAttr("threshold", 18)

# Don't do tracking or drift correction.
daop.changeAttr("radius", 0.0)
daop.changeAttr("drift_correction", 0)

# Save the changed parameters.
daop.toXMLFile("calibration.xml")


Analyze the calibration movie with 3D-DAOSTORM

In [ ]:
import os
import storm_analysis.daostorm_3d.mufit_analysis as mfit

if os.path.exists("calib.hdf5"):
    os.remove("calib.hdf5")
    
mfit.analyze("calib.tif", "calib.hdf5", "calibration.xml")


Check results with with overlay images.

In [ ]:
# Overlay image at z near zero.
import storm_analysis.jupyter_examples.overlay_image as overlay_image

overlay_image.overlayImage("calib.tif", "calib.hdf5", 40)


### Z calibration

First we will need a file containing the z-offsets for each frame. This file contains two columns, the first is whether or not the data in this frame should be used (0 = No, 1 = Yes) and the second contains the z offset in microns.

In [ ]:
import numpy

# In this simulation the z range went from -0.6 microns to 0.6 microns in 10nm steps.
z_range = dao3d_zcal.z_range
z_offsets = numpy.arange(-z_range, z_range + 0.001, 0.01)
valid = numpy.ones(z_offsets.size)

# Limit the z range to +- 0.4um.
mask = (numpy.abs(z_offsets) > 0.4)
valid[mask] = 0.0

numpy.savetxt("z_offsets.txt", numpy.transpose(numpy.vstack((valid, z_offsets))))


Plot Wx / Wy versus Z curves.

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot

# Change default figure size.
matplotlib.rcParams['figure.figsize'] = (8,6)

import storm_analysis.daostorm_3d.z_calibration as z_cal

[wx, wy, z, pixel_size] = z_cal.loadWxWyZData("calib.hdf5", "z_offsets.txt")

pyplot.scatter(z, wx, color = 'r')
pyplot.scatter(z, wy, color = 'b')
pyplot.show()

Now measure Z calibration curves. We'll do a second order fit, i.e. A,B will be fit, but not C,D.

Note - The fitting is not super robust, so you may have to play with `fit_order` and `p_start` to get it to work. Usually it will work for `fit_order = 0`, but then it might fail for `fit_order = 1` but succeed for `fit_order = 2`.

In [ ]:
#
# The function z_cal.calibrate() will perform all of these steps at once.
#

fit_order = 2
outliers = 3.0 # Sigma to be considered an outlier.

# Initial guess, this is optional, but might be necessary if your setup is
# significantly different from what storm-analysis expects.
#
# It can also help to boot-strap to higher fitting orders.
#
p_start = [3.2,0.19,0.3]

# Fit curves
print("Fitting (round 1).")
[wx_params, wy_params] = z_cal.fitDefocusingCurves(wx, wy, z, n_additional = 0, z_params = p_start)
print(wx_params)
p_start = wx_params[:3]

# Fit curves.
print("Fitting (round 2).")
[wx_params, wy_params] = z_cal.fitDefocusingCurves(wx, wy, z, n_additional = fit_order, z_params = p_start)
print(wx_params)
p_start = wx_params[:3]

# Remove outliers.
print("Removing outliers.")
[t_wx, t_wy, t_z] = z_cal.removeOutliers(wx, wy, z, wx_params, wy_params, outliers)

# Redo fit.
print("Fitting (round 3).")
[wx_params, wy_params] = z_cal.fitDefocusingCurves(t_wx, t_wy, t_z, n_additional = fit_order, z_params = p_start)

# Plot fit.
z_cal.plotFit(wx, wy, z, t_wx, t_wy, t_z, wx_params, wy_params, z_range = 0.4)

# This prints the parameter with the scale expected by 3D-DAOSTORM in the analysis XML file.
z_cal.prettyPrint(wx_params, wy_params, pixel_size = pixel_size)

Create a parameters file with these calibration values.

In [ ]:
# Load the parameters.
daop = params.ParametersDAO().initFromFile("example.xml")

# Update calibration parameters.
z_cal.setWxWyParams(daop, wx_params, wy_params, pixel_size)

# Do z fitting.
daop.changeAttr("do_zfit", 1)

# Set maximum allowed distance in wx, wy space that a point can be from the 
# calibration curve.
daop.changeAttr("cutoff", 2.0)

# Use a higher threshold as the Gaussian PSF is not a good match for our PSF model, so
# we'll get spurious peak splitting if it is too low.
daop.changeAttr("threshold", 12)

# Don't do tracking or drift correction as this movie is the same as the calibration
# movie, every frame has a different z value.
daop.changeAttr("radius", 0.0)
daop.changeAttr("drift_correction", 0)

# Save the changed parameters.
daop.toXMLFile("measure.xml")

### Analyze test movie with the z-calibration parameters.

In [ ]:
if os.path.exists("measure.hdf5"):
    os.remove("measure.hdf5")
    
mfit.analyze("measure.tif", "measure.hdf5", "measure.xml")


Plot Wx / Wy versus Z curves for data from the test movie.

In [ ]:
[wx, wy, z, pixel_size] = z_cal.loadWxWyZData("measure.hdf5", "z_offsets.txt")

pyplot.scatter(z, wx, color = 'r')
pyplot.scatter(z, wy, color = 'b')
pyplot.show()

Plot Wx versus Wy with the z calibration curve overlaid.

This can be useful for checking that your calibration curve matches your data.

In [ ]:

# Load Z calibration parameters.
m_params = params.ParametersDAO().initFromFile("measure.xml")

[wx_params, wy_params] = m_params.getWidthParams()
[min_z, max_z] = m_params.getZRange()

# Z range is in microns, want nanometers.
min_z = min_z * 1.0e+3
max_z = max_z * 1.0e+3

# Calculate fit z curve at high resolution
fz_wx_1 = z_cal.zcalib4(wx_params, numpy.arange(min_z, max_z + 1, 10))/dao3d_zcal.pixel_size
fz_wy_1 = z_cal.zcalib4(wy_params, numpy.arange(min_z, max_z + 1, 10))/dao3d_zcal.pixel_size

# Calculate fit z curve at 100nm resolution.
fz_wx_2 = z_cal.zcalib4(wx_params, numpy.arange(min_z, max_z + 1, 100))/dao3d_zcal.pixel_size
fz_wy_2 = z_cal.zcalib4(wy_params, numpy.arange(min_z, max_z + 1, 100))/dao3d_zcal.pixel_size

# Make figure.
fig = pyplot.figure(figsize = (8,8))
pyplot.scatter(wx, wy, marker = ".")
pyplot.scatter(fz_wx_2, fz_wy_2, marker = "o", s = 120, edgecolor = "black", facecolor = 'none', linewidths = 2)
pyplot.plot(fz_wx_1, fz_wy_1, color = "black", linewidth = 2)
pyplot.xlim(2,10)
pyplot.ylim(2,10)
pyplot.xlabel("Wx (pixels)")
pyplot.ylabel("Wy (pixels)")
pyplot.show()

Check how well we did at fitting Z.

In [ ]:
import storm_analysis.sa_library.sa_h5py as saH5Py

# Create numpy arrays with the real and the measured z values.
measured_z = numpy.array([])
real_z = numpy.array([])

with saH5Py.SAH5Reader("measure.hdf5") as h5:
    for fnum, locs in h5.localizationsIterator(fields = ["category", "z"]):
        
        # The z fit function will place all the localizations that are too
        # far from the calibration curve into category 9.
        mask = (locs["category"] != 9)
        z = locs["z"][mask]
        measured_z = numpy.concatenate((measured_z, z))
        real_z = numpy.concatenate((real_z, numpy.ones(z.size)*z_offsets[fnum]))
        
# Plot
fig = pyplot.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
ax.scatter(real_z, measured_z, s = 4)
ax.plot([-1.0,1.0],[-1.0,1.0], color = 'black', linewidth = 2)
ax.axis("equal")
ax.axis([-0.5, 0.5, -0.5, 0.5])
pyplot.xlabel("Actual Z (um)")
pyplot.ylabel("Measured Z (um)")

Change the tolerance for the distance from the calibration curve and redo the Z fit.

In [ ]:
import shutil
import storm_analysis.sa_utilities.fitz_c as fitz_c
import storm_analysis.sa_utilities.std_analysis as std_ana

m_params = params.ParametersDAO().initFromFile("measure.xml")

[wx_params, wy_params] = m_params.getWidthParams()
[min_z, max_z] = m_params.getZRange()

# Make a copy of the .hdf5 file as this operation will change it in place.
shutil.copyfile("measure.hdf5", "measure_copy.hdf5")

m_params.changeAttr("cutoff", 0.2)
print("cutoff is", m_params.getAttr("cutoff"))

# Re-fit z parameters.
fitz_c.fitz("measure_copy.hdf5", m_params.getAttr("cutoff"),
            wx_params, wy_params, min_z, max_z, m_params.getAttr("z_step"))

# Mark out of range peaks as category 9. The range is specified by the min_z and max_z parameters.
std_ana.zCheck("measure_copy.hdf5", m_params)

In [ ]:

# Create numpy arrays with the real and the measured z values.
measured_z = numpy.array([])
real_z = numpy.array([])

with saH5Py.SAH5Py("measure_copy.hdf5") as h5:
    for fnum, locs in h5.localizationsIterator(fields = ["category", "z"]):

        # The z fit function will place all the localizations that are too
        # far from the calibration curve into category 9.
        mask = (locs["category"] != 9)
        z = locs["z"][mask]
        measured_z = numpy.concatenate((measured_z, z))
        real_z = numpy.concatenate((real_z, numpy.ones(z.size)*z_offsets[fnum]))
        
# Plot
fig = pyplot.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
ax.scatter(real_z, measured_z, s= 4)
ax.plot([-1.0,1.0],[-1.0,1.0], color = 'black', linewidth = 2)
ax.axis("equal")
ax.axis([-0.5, 0.5, -0.5, 0.5])
pyplot.xlabel("Actual Z (um)")
pyplot.ylabel("Measured Z (um)")